In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re
from io import StringIO

# 1.0 Scraping Data

## 1.1 Function for Page

In [2]:
def page_scrape(startplayerdf,startstatdf):
    """
    Function to pull all relevent data in player page and add the information to the base dataframe
    There are 2 data frames:
        The player Data Frame with name, height, draft nbr etc 
        The stats data frame with all of the adv stats by year for the player
        They are connected through the player name
    
    """
    
    #Name, Height, Weight Come directly from xpath lookups
    name = driver.find_element_by_xpath('//h1[@itemprop="name"]').text
    
    height = driver.find_element_by_xpath('//span[@itemprop="height"]').text
    height = height.split('-')
    height = int(height[0])+(int(height[1])/12)
    
    weight = driver.find_element_by_xpath('//span[@itemprop="weight"]').text
    weight = int(weight[:-2])
    
    #Draft Number, Draft year and Recruit number come from a larger info section, this breaks up the info section
    info = driver.find_element_by_xpath('//div[@id="info"]').text
    info = info.split('\n')
    draft_regex = re.compile('Draft:')
    draftstr = [x  for x in info if draft_regex.search(x)]
    draftstr = draftstr[0].split(',')
    
    draftnbr = draftstr[2].strip().split(' ')
    draftnbr = draftnbr[0]
    draftnbr = int(draftnbr[:-2])
    
    draftyr = draftstr[3].strip().split(' ')
    draftyr = int(draftyr[0])
    
    try: #Not all players have recruit number
        recruit_regex = re.compile('Recruiting')
        recruitstr = [x  for x in info if recruit_regex.search(x)]    
        recruitstr = recruitstr[0].split(' ')
        recruitnbr = recruitstr[-1]
        recruitnbr = int(recruitnbr[1:-1])
    except:
        recruitnbr = 0
    
    #creates a new data frame for player info to append to the starting player df
    dfscrapeinfo = pd.DataFrame(index=[0])
    dfscrapeinfo['player'] = name
    dfscrapeinfo['height'] = height
    dfscrapeinfo['weight'] = weight
    dfscrapeinfo['draftnbr']  = draftnbr
    dfscrapeinfo['draftyr'] = draftyr
    dfscrapeinfo['recruitnbr'] = recruitnbr
    
    startplayerdf = startplayerdf.append(dfscrapeinfo)
    
    
    #Gets the advanced stats table into a df then appends on to starting stats df
    advstatsxpath = driver.find_element_by_xpath('//table[@id="advanced"]').text
    advstatsxpath = advstatsxpath.replace('  ','')
    dfadvstats = pd.read_csv(StringIO(advstatsxpath),sep=' ')
    dfadvstats = dfadvstats.dropna()
    dfadvstats['player'] = name
    
    startstatdf = startstatdf.append(dfadvstats)
    
    #prints last completed name so I can error test / log progress
    return print(name)
    